# 第06章 — 自律エージェント

- ステアリング行動は望ましい速度変化（シーク、逃避、到着、徘徊）を合成する。
- ボイドでは分離・整列・結合の三つの力を重み付けする。
- 操舵力と速度を制限すると動きが滑らかになる。
- 経路追従では進行方向の先をターゲットとするセグメントとして扱う。

試してみよう: max_forceを下げて鈍い反応にしたり、ノイズを足して徘徊を試す。

In [ ]:
# ステアリング用の2Dベクトルクラス
import random

class Vector:
    def __init__(self, x=0.0, y=0.0):
        self.x, self.y = float(x), float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, k):
        return Vector(self.x * k, self.y * k)

    __rmul__ = __mul__

    def mag(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5

    def normalize(self):
        m = self.mag()
        if m == 0:
            return Vector()
        return self * (1.0 / m)

    def limit(self, max_mag):
        m = self.mag()
        if m > max_mag:
            return self.normalize() * max_mag
        return self

    def __repr__(self):
        return f"Vector({self.x:.2f}, {self.y:.2f})"


In [ ]:
# シーク行動を持つエージェントクラス
class Agent:
    def __init__(self, pos):
        self.pos = Vector(*pos)
        self.vel = Vector(random.uniform(-1, 1), random.uniform(-1, 1)).limit(2)
        self.max_speed = 2.5
        self.max_force = 0.15

    def seek(self, target):
        desired = (target - self.pos).normalize() * self.max_speed
        steer = (desired - self.vel).limit(self.max_force)
        return steer

    def apply(self, force):
        self.vel = self.vel + force
        self.vel = self.vel.limit(self.max_speed)
        self.pos = self.pos + self.vel


### 追加例: 徘徊（wander）ステアリング

In [ ]:
# 徘徊ステアリングのためのワンダラーを定義
import math
import random

class Wanderer:
    def __init__(self):
        self.pos = Vector(0, 0)
        self.vel = Vector(1, 0)
        self.theta = 0.0
        self.max_speed = 2.0
        self.max_force = 0.2

    def wander(self):
        self.theta += random.uniform(-0.5, 0.5)
        circle_dir = Vector(math.cos(self.theta), math.sin(self.theta))
        desired = (self.vel.normalize() + circle_dir).normalize() * self.max_speed
        steer = (desired - self.vel).limit(self.max_force)
        self.vel = (self.vel + steer).limit(self.max_speed)
        self.pos = self.pos + self.vel
        return self.pos, self.vel


In [ ]:
# 徘徊を数ステップ回して位置と速度を表示
w = Wanderer()
for i in range(10):
    pos, vel = w.wander()
    print(f"step {i}: pos={pos} vel={vel}")
